In [3]:
import json
import gzip
import gc
import math
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from tqdm.notebook import trange, tqdm
from sklearn.metrics import ndcg_score
#from challenge_metric import ndcg_score
tqdm.pandas()


C:\ProgramData\Anaconda3\envs\env-meli\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def jl_to_list(filename):
    output=[]
    with gzip.open(filename, 'rb') as f:
        for line in tqdm(f):
            output.append(json.loads(line))
    return output

def dominios_visitados(row, max_views=15):
    domains = Counter()
    viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
    if len(viewed) > max_views:
        viewed = viewed[:15]
    for iten in viewed:
        domain = metadata[item]['domain_id']
        domains[domain] += 1
    return domains

def top_items(domain, k=10):
    top = ventas_x_dominio[domain]
    top = Counter(top)
    top = top.most_common(k)
    return [x[0] for x in top]

def top_by_best_domain(row, k=10):
    viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
    if len(viewed) == 0:
        return random.choices(all_items, k=k)
    domain = dominios_visitados(row)
    domain = domain.most_common(1)[0][0]
    return top_items(domain, k=k)

def last_viewed(row, k=10):
    viewed = [ev for ev in row['user_history'] if ev['event_type'] == 'view']
    viewed = sorted(viewed, key=lambda x: x['event_timestamp'], reverse=True)
    viewed = [ev['event_info'] for ev in viewed]
    recom = []
    for item in viewed:
        if item not in recom:
            recom.append(item)
            
    if len(recom) > 10:
        recom = recom[:10]
        
    if len(recom) == 10:
        return recom
    
    k = 10 - len(recom)
    relleno = random.choices(all_items, k=k)
    return recom + relleno

def get_item_scores(row):
    item_scores = defaultdict(int)
    for ev in row['user_history']:
        if ev['event_type'] == 'view':
            for k, v in vistas_compras[int(ev['event_info'])].items():
                item_scores[k]+=v
    return Counter(item_scores)

def vc_reco(row):
    reco=[]
    scores = get_item_scores(row)
    most_common = scores.most_common()
    for item, score in most_common:
        reco.append(item)
        if len(reco) == 10:
            return reco
    k = 10 - len(reco)
    relleno = random.choices(all_items, k=k)
    return reco + relleno

def get_ndcg_score(y_true, y_pred, item_data, n_predictions=10):
    #score = ndcg_score(y_true, y_pred, item_data, n_predictions=n_predictions)
    score = ndcg_score(y_true, y_pred)
    print(f'Your score is: {score}')

In [5]:
train_filename = 'train_dataset.jl.gz'
test_filename = 'test_dataset.jl.gz'
item_data_filename = 'item_data.jl.gz'

samples = 50000
rows = jl_to_list(train_filename)
if samples:
    rows = rows[:samples]

rows_train, rows_test = train_test_split(rows, test_size=0.2, random_state=42)

In [6]:
item_data = jl_to_list(item_data_filename)
metadata = {x['item_id']:x for x in item_data}
all_items = list(metadata.keys())

In [7]:
test_rows = jl_to_list(test_filename)
test_rows[0]

{'user_history': [{'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:31:47.705-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:35:04.724-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:37:35.532-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:38:54.680-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:40:26.904-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:40:35.707-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:41:07.467-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T18:41:47.445-0400',
   'event_type': 'view'},
  {'event_info': 1572239,
   'event_timestamp': '2019-09-26T19:03:34.256-0400',
   'event_type': 'view'},
  {'event_info': 1194894,
   '

In [8]:
ventas_x_dominio = defaultdict(lambda: defaultdict(int))

for row in tqdm(rows_train):    
    viewed = [ev['event_info'] for ev in row['user_history'] if ev['event_type'] == 'view']
    for item in viewed:
        domain = metadata[item]['domain_id']
        ventas_x_dominio[domain][item]+=1


In [9]:
dominios_visitados(rows_train[2])

Counter({'MLB-CAR_AC_DASHBOARD_VENTS': 15})

In [10]:
top_items('MLB-CAR_AC_DASHBOARD_VENTS')

[154508,
 298530,
 259916,
 508546,
 1728938,
 1540693,
 1984130,
 1232794,
 2080778,
 909392]

In [11]:
top_by_best_domain(rows_train[0])

[154508,
 298530,
 259916,
 508546,
 1728938,
 1540693,
 1984130,
 1232794,
 2080778,
 909392]

In [12]:
y_pred = []
for row in tqdm(rows_test):
    recom = top_by_best_domain(row)
    y_pred.append(recom)

In [13]:
y_true = [row['item_bought'] for row in rows_test]

In [18]:
get_ndcg_score(np.asarray(y_true), y_pred, item_data, n_predictions=10)

ValueError: Only ('multilabel-indicator', 'continuous-multioutput', 'multiclass-multioutput') formats are supported. Got multiclass instead

In [20]:

y_pred = []
for row in tqdm(test_rows):
    recom = top_by_best_domain(row)
    y_pred.append(recom)
pd.DataFrame(y_pred).to_csv('submission.csv')

In [ ]:
last_viewed(rows[0])

In [ ]:
y_pred = []
for row in tqdm(rows_test):
    recom = last_viewed(row)
    y_pred.append(recom)

In [ ]:
get_ndcg_score(y_true, y_pred, item_data, n_predictions=10)

In [ ]:
vistas_compras = defaultdict(lambda: defaultdict(int))
for row in tqdm(rows_train):
    for ev in row['user_history']:
        if ev['event_type'] == 'view':
            vistas_compras[int(ev['event_info'])][int(row['item_bought'])]+=1

In [ ]:
y_pred = []
for row in tqdm(rows_test):
    recom = vc_reco(row)
    y_pred.append(recom)

In [ ]:
get_ndcg_score(y_true, y_pred, item_data, n_predictions=10)

In [ ]:
#baselines se pueden usar como generadores


In [1]:
# import required package 
from sklearn.metrics import ndcg_score, dcg_score 
import numpy as np 

# Releveance scores in Ideal order 
true_relevance = np.asarray([[3, 2, 1, 0, 0]]) 

# Releveance scores in output order 
relevance_score = np.asarray([[3, 2, 0, 0, 1]]) 

# DCG score 
dcg = dcg_score(true_relevance, relevance_score) 
print("DCG score : ", dcg) 

# IDCG score 
idcg = dcg_score(true_relevance, true_relevance) 
print("IDCG score : ", idcg) 

# Normalized DCG score 
ndcg = dcg / idcg 
print("nDCG score : ", ndcg) 

# or we can use the scikit-learn ndcg_score package 
print("nDCG score (from function) : ", ndcg_score( 
	true_relevance, relevance_score)) 


DCG score :  4.670624189796882
IDCG score :  4.761859507142915
nDCG score :  0.980840401274087
nDCG score (from function) :  0.980840401274087


In [19]:
relevance_score

array([[3, 2, 0, 0, 1]])